In [ ]:
import pandas as pd

# 데이터 획득
주성분 분석을 실습하기 위한 데이터를 만들어보겠습니다.
사람들의 하루 식습관 데이터를 만든 후, 주성분 분석을 사용하여 데이터를 시각화해보도록 하겠습니다.
먼저 칼로리, 아침, 점심, 저녁, 운동 횟수, 그리고 체형이라는 특징을 가진 데이터프레임을 생성합니다.

In [ ]:
df = pd.DataFrame(columns=['calory', 'breakfast', 'lunch', 'dinner', 'exercise', 'body_shape'])

10명의 가공 데이터를 아래처럼 만들도록 하겠습니다.

In [ ]:
df.loc[0] = [1200, 1, 0, 0, 2, 'Skinny']
df.loc[1] = [2800, 1, 1, 1, 1, 'Normal']
df.loc[2] = [3500, 2, 2, 1, 0, 'Fat']
df.loc[3] = [1400, 0, 1, 0, 3, 'Skinny']
df.loc[4] = [5000, 2, 2, 2, 0, 'Fat']
df.loc[5] = [1300, 0, 0, 1, 2, 'Skinny']
df.loc[6] = [3000, 1, 0, 1, 1, 'Normal']
df.loc[7] = [4000, 2, 2, 2, 0, 'Fat']
df.loc[8] = [2600, 0, 2, 0, 0, 'Normal']
df.loc[9] = [3000, 1, 2, 1, 1, 'Fat']

가공 데이터를 확인해보도록 하겠습니다.

In [ ]:
df.head(10)

# 데이터 전처리
주성분 분석을 위해 데이터 전처리 과정이 필요합니다.  
우선 가공 데이터에 데이터의 특징과 클래스가 함께 있으므로,  
데이터의 특징만으로만 구성된 X 데이터프레임을 생성합니다.

In [ ]:
X = df[['calory', 'breakfast', 'lunch', 'dinner', 'exercise']]

In [ ]:
X.head(9)

body_shape는 데이터 특징들로 구별된 클래스입니다.  
Y 데이터프레임을 생성하여 데이터별 클래스를 저장합니다.  
body_shape 정보는 추후 차원 축소된 데이터를 시각화할 때,  
데이터의 정보가 충분히 유지되었는 지 확인하기 위해 사용합니다.

In [ ]:
Y = df[['body_shape']]

In [ ]:
Y.head(10)

데이터의 특징들이 수치이기는 하지만, 각각의 범위가 다르므로, 범위를 통일시켜줍니다.  
데이터를 정규화함으로써, 머신러닝 알고리즘이 범위가 넓은 데이터에 중점을 두는 현상이 줄고,  
정확도가 높아집니다.

In [ ]:
from sklearn.preprocessing import StandardScaler
x_std = StandardScaler().fit_transform(X)

In [ ]:
x_std

# 공분산 행렬 구하기
주성분 분석을 하기 위해 가장 먼저 특징들의 공분산 행렬을 구해야합니다.  
아래의 코드를 실행하여 간단하게 공분산 행렬을 구할 수 있습니다.

In [ ]:
import numpy as np

In [ ]:
features = x_std.T 
covariance_matrix = np.cov(features)
print(covariance_matrix)

# 고유값과 고유벡터 구하기
공분산 행렬에서 고유값(eigen value)과 고유벡터(eigen vector)를 아래의 코드를 실행하여 구할 수 있습니다.

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)

In [ ]:
print('Eigenvectors \n%s' %eig_vecs)

In [ ]:
print('\nEigenvalues \n%s' %eig_vals)

아래의 코드를 실행하여 가장 큰 고유벡터의 정보 함유량을 볼 수 있습니다.  

In [ ]:
# We reduce dimension to 1 dimension, since 1 eigenvector has 73% (enough) variances
eig_vals[0] / sum(eig_vals)

가장 큰 고유 벡터가 73%의 정보를 함유하고 있습니다.  
5차원 데이터를 가장 큰 고유 벡터(1차원)로 차원 축소해보겠습니다.

# 5차원 데이터를 고유벡터로 사영시키기
5차원 데이터를 가장 큰 고유벡터로 옮겨보도록하겠습니다.  
a벡터를 b벡터에 사영할 때 공식 = dot(a,b) / magnitude of b

In [ ]:
projected_X = x_std.dot(eig_vecs.T[0]) / np.linalg.norm(eig_vecs.T[0]) 

1차원 벡터로 사영된 결과를 출력해봅니다.

In [ ]:
projected_X

# 시각화
1차원으로 축소된 데이터를 시각화하기 위해, 팬더스 데이터프레임에 데이터를 담습니다.  
주성분(PC1)을 x축으로 하고, 1차원 데이터이므로 y축은 0으로 통일시킵니다.  
데이터의 쉬운 이해를 위해 클래스를 데이터 마지막 컬럼에 포함시킵니다.

In [ ]:
result = pd.DataFrame(projected_X, columns=['PC1'])
result['y-axis'] = 0.0
result['label'] = Y

아래 코드를 실행하여 데이터프레임을 확인합니다.

In [ ]:
result.head(10)

데이터를 표로 보는 것보다 챠트에서 보는 게 훨씬 이해가 쉽습니다.  
아래의 코드를 실행하여 챠트에 차원 축소된 데이터를 시각화해봅니다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.lmplot('PC1', 'y-axis', data=result, fit_reg=False,  # x-axis, y-axis, data, no line
           scatter_kws={"s": 50}, # marker size
           hue="label") # color

# title
plt.title('PCA result')

5차원이었던 데이터를 1차원으로 줄여서 시각화가 가능해졌습니다.  
또한 체형별 군집이 잘 형성된 것으로 보아, 차원 축소에도 불구하고 충분한 정보가 유지된 것을,  
눈으로 확인 가능합니다.

# 보너스 - scikit-learn을 사용한 주성분 분석 구현
sklearn을 사용하여 간단하게 주성분 분석(PCA)를 구현할 수 있습니다.

아래 코드를 실행하여, 1차원으로 데이터를 차원 축소합니다.

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=1)
sklearn_pca_x = pca.fit_transform(x_std)

아래의 코드를 실행하여 시각화합니다.
우리가 직접 구현한 코드와 동일한 결과를 확인하실 수 있습니다.

In [ ]:
sklearn_result = pd.DataFrame(sklearn_pca_x, columns=['PC1'])
sklearn_result['y-axis'] = 0.0
sklearn_result['label'] = Y

sns.lmplot('PC1', 'y-axis', data=sklearn_result, fit_reg=False,  # x-axis, y-axis, data, no line
           scatter_kws={"s": 50}, # marker size
           hue="label") # color